In [1]:
import torch
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler
from torch.utils.data import Dataset
from torchvision import transforms


def best_transform(data):
    """
    Функция с ироничным названием и функционал,
    целью существования которой является ее
    применение к уже обработанным данным, но
    для структуры датасета необходима функция
    преобразования
    """
    return data


def mnist_transform():
    """
    Функция предназаченная для примения
    в датасете MNIST
    :return: функция преобразования
    """
    transform = transforms.Compose([
        transforms.ToPILImage(),
        # transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.1307,), (0.3081,))
    ])
    return transform


class TrainDataset(Dataset):
    """
    Класс, описывающий набор данных для обучения
    """
    def __init__(self, features, labels, Transform):
        self.x = features
        self.y = labels
        self.transform = Transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.transform(self.x[index]), self.y[index]


def get_mnist_data(df, Transform=best_transform):
    """
    Предобработка,для MNIST датасета
    """
    x_features = df.iloc[:, 1:].values
    y_labels = df.label.values
    x_features = x_features.reshape(-1, 1, 28, 28)
    x_features = np.uint8(x_features)
    x_features = torch.from_numpy(x_features)
    y_labels = torch.from_numpy(y_labels)
    return TrainDataset(x_features, y_labels, Transform)


def resize_data(X, y, time_steps=1, step=1):
    """
    Разбиение набора данных на пересекающие фреймы
    :param X: матрица признаков
    :param y: целевое значение
    :param time_steps: размер временного окна
    :param step: шаг между началами окон
    :return:
    """
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1)


def get_smartiliser_data(df, Transform=best_transform):
    """
    Функция для предобработки смартилайзеровских данных
    :param df: DataFrame
    :param Transform: функция преобразования
    :return: TrainDataset
    """
    # Параметры временного окна
    time_steps = 40
    step = 10
    # Снижение частоты дискретизации
    df = df[::2]
    # Отбор классов для обучения
    df = df[(df.activityMode == 5) | (df.activityMode == 6) | (df.activityMode == 7)]
    y_labels = df.activityMode - 5
    # Отбор признаков для обучения
    scale_columns = ['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
    df = df[scale_columns]
    # Применение надежного скалера
    scaler = RobustScaler()
    scaler = scaler.fit(df)
    df.loc[:, scale_columns] = scaler.transform(df[scale_columns].to_numpy())
    x_features = df
    # Преобразование временных рядов в набор временных окон
    x_features, y_labels = resize_data(x_features, y_labels, time_steps, step)
    x_features = x_features.reshape(len(x_features), -1)
    # Приведение матрицы признаков к float 32
    x_features = np.float32(x_features)
    return TrainDataset(x_features, y_labels, Transform)


def get_data_loader(path, batch_size, data_name='MNIST'):
    """
    Функция для получения необходимого DataLoader
    :param path: путь к CSV файлу
    :param batch_size: размер пакета
    :param data_name: название набора данных
    :return: DataLoader
    """
    # Чтение CSV файла
    train_df = pd.read_csv(path)
    # Словарь предобработок
    preprocces = {
        'MNIST': get_mnist_data,
        'smartiliser': get_smartiliser_data
    }
    # Словарь функций преобразования
    transform = {
        'MNIST': mnist_transform(),
        'smartiliser': best_transform,
    }
    train_loader = torch.utils.data.DataLoader(
        preprocces[data_name](train_df, transform[data_name]),
        batch_size=batch_size, shuffle=True)

    return train_loader


In [2]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


loss_functions = {
    ''
}

class MiningSettings:
    """
    Класс для сериализации настроек алгоритма для
    обучения нейронной сети
    """
    def __init__(self, algorithm, loss_function, epochs, learning_rate, momentum, batch_size):
        self.algorithm = algorithm
        self.loss_function = loss_function
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size

class UniversalNet(nn.Module):
    """
    Класс универсальной модели
    """
    def __init__(self, layers, activations, normalizations):
        super(UniversalNet, self).__init__()
        self.number_of_layers = len(layers)
        self.layers = nn.ModuleList(layers)
        self.activations = activations
        self.normalizations = normalizations

    def forward(self, x):
        """
        Метод для прямого прохода по нейронной сети
        :param x: данные для прохода
        :return: данные после прохода
        """
        for i in range(self.number_of_layers):
            # Прохождения слоя
            x = self.layers[i](x)
            # Применение функции активации
            x = self.activations[i](x)
            # Применение нормализации
            x = self.normalizations[i](x)
        return x


def evaluate(test_loader, criterion, net):
    """
    Функция оценки точности модели
    :param test_loader: DataLoader
    :param criterion: функция потерь
    :param net: UniversalNet
    :return: точность и ошибка
    """
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 40 * 6)
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).data.item()
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f})\n'.format(
        test_loss, correct, len(test_loader.dataset),
        correct / len(test_loader.dataset)))


def fit(net, train_loader, epochs, criterion, optimizer):
    """
    Функция, тренировки модели
    :param net: UniversalNet
    :param train_loader: DataLoader
    :param epochs: количество эпох
    :param criterion: функция потерь
    :param optimizer: метод оптимизации
    :return: UniversalNet
    """
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            data = data.view(-1, 40*6)

            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            # print(loss.data)
            #if batch_idx % 100 == 0:
            #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #         epoch, batch_idx * len(data), len(train_loader.dataset),
            #                100. * batch_idx / len(train_loader), loss.item()))
    return net


def train_evaluate(net, path='D:/FL_client/data/MNIST/train.csv', settings='nothing', data_name='MNIST'):
    """
    Функция для обучения и оценки точности модели
    :param net: UniversalNet
    :param path: путь к CSV файлу
    :param settings: MiningSettings
    :param data_name: название набора данных
    :return: UniversalNet
    """
    # Загрузка набора данных
    train_loader = get_data_loader(path, settings.batch_size, data_name=data_name)
    # Создание объекта для метода оптимизации
    optimizer = optim.SGD(net.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
    # Создание объекта для функции потерь
    criterion = settings.loss_function()
    # Вызов функции обучения
    net = fit(net, train_loader, settings.epochs, criterion, optimizer)
    # Вызов функции оценки точности модели
    evaluate(train_loader, criterion, net)
    return net, len(train_loader)

In [5]:
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
settings = MiningSettings(
        algorithm='SGD',
        loss_function=nn.NLLLoss,
        epochs=5,
        learning_rate=0.1,
        momentum=0.9,
        batch_size=10
)
net = UniversalNet(layers, activations, normalizations)
net = train_evaluate(net=net, path='D:\FL_client\data\smartilizer\Video-11-15-40-560.csv', data_name='smartiliser', settings=settings)


d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0123, Accuracy: 103341/106753 (0.968)



In [6]:
test = get_data_loader('D:\FL_client\data\smartilizer\Video-16-8-9-212.csv', batch_size=10, data_name='smartiliser')

In [7]:
criterion = settings.loss_function()

In [8]:
evaluate(test, criterion, net[0])

d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0666, Accuracy: 15391/17767 (0.866)



In [9]:
import os
path = 'D:\\FL_client\\data\\smartilizer'
datasets = []
for folder in os.listdir(path)[1:6]:
    path_file = os.path.join(path, folder)
    datasets.append(get_data_loader(path_file, batch_size=10, data_name='smartiliser'))


In [ ]:
os.listdir(path)

In [17]:
# 1 model
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
net_1 = UniversalNet(layers, activations, normalizations)
optimizer = optim.SGD(net_1.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
net_1 = fit(net_1, datasets[0], 5, criterion, optimizer)

d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [16]:
# 2 model
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
net_2 = UniversalNet(layers, activations, normalizations)
optimizer = optim.SGD(net_2.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
net_2 = fit(net_2, datasets[1], 5, criterion, optimizer)

d:\fl_client\venv\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 200])

In [ ]:
# 3 model
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
net_3 = UniversalNet(layers, activations, normalizations)
optimizer = optim.SGD(net_3.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
net_3 = fit(net_3, datasets[2], 5, criterion, optimizer)

In [ ]:
# 4 model
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
net_4 = UniversalNet(layers, activations, normalizations)
optimizer = optim.SGD(net_4.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
net_4 = fit(net_4, datasets[3], 5, criterion, optimizer)

In [ ]:
# 5 model
layers = [
        nn.Linear(240, 200),
        nn.Linear(200, 200),
        nn.Linear(200, 100),
        nn.Linear(100, 3)
]
activations = [
        F.relu,
        F.relu,
        F.relu,
        F.log_softmax
]
normalizations = [
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(200),
        nn.BatchNorm1d(100),
        lambda x: x
]
net_5 = UniversalNet(layers, activations, normalizations)
optimizer = optim.SGD(net_5.parameters(), lr=settings.learning_rate, momentum=settings.momentum)
net_5 = fit(net_5, datasets[4], 5, criterion, optimizer)